In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch.nn as nn
import torch.optim as optim
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler

In [3]:
class HousingDataset(Dataset):
    def __init__(self, csv_file):

        df = pd.read_csv(csv_file)
        yes_no = ['mainroad','guestroom','basement','hotwaterheating','airconditioning','prefarea']
        for c in yes_no:
            if c in df: df[c] = df[c].map({'yes':1,'no':0})

        df = pd.get_dummies(df, columns=['furnishingstatus'], dtype=int)

        self.y = df['price'].values.astype('float32').reshape(-1,1)
        X = df.drop(columns=['price']).values.astype('float32')
        self.scaler = StandardScaler()
        self.X = self.scaler.fit_transform(X)
        self.X = torch.from_numpy(self.X)
        self.y = np.log1p(df['price'].values.astype('float32')).reshape(-1,1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [4]:
path = 'Housing.csv'
ds = HousingDataset(path)
loader = DataLoader(ds, batch_size = 4, shuffle = True)

In [5]:
next(iter(loader))

[tensor([[ 0.1381, -1.3089, -0.5702, -0.9294,  0.4056, -0.4653, -0.7345, -0.2193,
          -0.6791, -0.8057, -0.5540,  1.7008, -0.8449, -0.6964],
         [-0.5122, -1.3089, -0.5702, -0.9294,  0.4056, -0.4653, -0.7345, -0.2193,
          -0.6791,  0.3560, -0.5540, -0.5879, -0.8449,  1.4359],
         [-0.4961,  0.0473, -0.5702, -0.9294,  0.4056,  2.1491,  1.3614, -0.2193,
          -0.6791,  1.5177, -0.5540, -0.5879,  1.1836, -0.6964],
         [ 0.5578, -1.3089, -0.5702, -0.9294,  0.4056, -0.4653, -0.7345, -0.2193,
          -0.6791, -0.8057, -0.5540,  1.7008, -0.8449, -0.6964]]),
 tensor([[15.1247],
         [15.0170],
         [15.2834],
         [15.2671]])]

In [6]:
class Model(nn.Module):
    def __init__(self, in_feature):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_feature, 1),
            nn.ReLU()
        )

    def forward(self, x):
        return self.net(x)

model = Model(ds.X.shape[1])

In [7]:
crit = nn.MSELoss()
optimizer = optim.Adam